### 1. 전처리, 피처공학

In [3]:
# # Data Preprocessing
import pandas as pd
import numpy as np

data = pd.read_csv('../DATA/raw_2023051820231018_경대기업맞춤형.csv')
print('기본 데이터 shape :', data.shape)

# 2) scale_pv < 5, E_scr_pv == 8, k_rpm_pv > 50
data = data[data['scale_pv'] < 5]  # 약 1800개 제거
print('scale_pv < 5 shape :', data.shape)
data = data[data['E_scr_pv'] == 8]  # 약 3800개 제거
print('E_scr_pv == 8 shape :', data.shape)
data = data[data['k_rpm_pv'] > 50] # 약 170개 제거
print('k_rpm_pv > 100 shape :', data.shape)

# 3) E_scr_sv, c_temp_sv, n_temp_sv, s_temp_sv, k_rpm_sv, n_temp_sv 제거
data.drop(['E_scr_sv', 'E_scr_pv', 'c_temp_sv', 's_temp_sv', 'k_rpm_sv', 'n_temp_sv', 'Unnamed: 12'], axis=1, inplace=True)

data['time'] = pd.to_datetime(data['time'])
oct_data = data[data['time'].dt.month == 10]
oct_data = oct_data.drop('time', axis=1)
print('oct_data shape :', oct_data.shape)

train_data = data[data['time'].dt.month != 10]
train_data = train_data.drop('time', axis=1)
print('train_data shape :', train_data.shape)

기본 데이터 shape : (235413, 13)
scale_pv < 5 shape : (233676, 13)
E_scr_pv == 8 shape : (229983, 13)
k_rpm_pv > 100 shape : (229810, 13)
oct_data shape : (29651, 5)
train_data shape : (200159, 5)


### 스케일링 X. 증강 X

In [7]:
# train, test 분리
from sklearn.model_selection import train_test_split

# 2 < scale_pv < 4
train_data_2_4 = train_data[(train_data['scale_pv'] > 2) & (train_data['scale_pv'] < 4)]

Feature_train = train_data_2_4.drop('scale_pv', axis=1)
Target_train = train_data_2_4['scale_pv']

X_train, X_test, y_train, y_test = train_test_split(Feature_train, Target_train, test_size=0.2, random_state=42) 


In [22]:
# 스케일링 없이 데이터 학습 및 분석
# pipeline
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Lasso, Ridge, BayesianRidge, ElasticNet, SGDRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score


# 모델 리스트
models = [
    ('LinearRegression', LinearRegression()),
    ('Lasso', Lasso()),
    ('Ridge', Ridge()),
    ('BayesianRidge', BayesianRidge()),
    ('ElasticNet', ElasticNet()),
    ('SGDRegressor', SGDRegressor()),
    ('RandomForestRegressor', RandomForestRegressor()),
    ('GradientBoostingRegressor', GradientBoostingRegressor()),
    ('AdaBoostRegressor', AdaBoostRegressor()),
    ('LGBMRegressor', LGBMRegressor())
]

# [ cross_val_score 사용은 보류]
# # 모델별 성능 확인
# for name, model in models:
#     pipeline = Pipeline([
#         ('scaler', StandardScaler()),
#         ('model', model)
#     ])
#     scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='neg_mean_absolute_error')
#     print(f'{name} : {scores.mean()}')
    
# 모델별 성능 확인
score_df = pd.DataFrame(columns=['model', 'MAE', 'MAPE', 'R2'])

for name, model in models:
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('model', model)
    ])
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    # print(f'[ {name} ]')
    # print('MAE :', mean_absolute_error(y_test, y_pred))
    # print('MAPE :', mean_absolute_percentage_error(y_test, y_pred)*100)
    # print('R2 :', r2_score(y_test, y_pred))
    
    diff = y_test - y_pred
    
    # score_df에 저장 : concat
    score_df = pd.concat(
        [score_df, 
         pd.DataFrame(
            [
              [name, mean_absolute_error(y_test, y_pred), 
              mean_absolute_percentage_error(y_test, y_pred)*100, 
              r2_score(y_test, y_pred),
                diff.mean(), diff.std(), diff.max(), diff.min(), diff.median()]
            ],
            columns=['model', 'MAE', 'MAPE', 'R2', 'dif_mean', 'dif_std', 'dif_max', 'dif_min', 'dif_median']
            )
        ]
    )
    
score_df

C:\Users\wjs31\AppData\Local\Temp\ipykernel_17876\1885545908.py:54: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  score_df = pd.concat(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000099 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 212
[LightGBM] [Info] Number of data points in the train set: 29376, number of used features: 4
[LightGBM] [Info] Start training from score 3.041735
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\wjs31\.conda\envs\EXAM_MML\lib\site-packages\IPython\core\formatters.py", line 223, in catch_format_error
    r = method(self, *args, **kwargs)
  File "c:\Users\wjs31\.conda\envs\EXAM_MML\lib\site-packages\IPython\core\formatters.py", line 344, in __call__
    return method()
  File "c:\Users\wjs31\.conda\envs\EXAM_MML\lib\site-packages\pandas\core\frame.py", line 1175, in _repr_html_
    # check whether repr fits horizontal by actually checking
  File "c:\Users\wjs31\.conda\envs\EXAM_MML\lib\site-packages\pandas\io\formats\format.py", line 1074, in to_html
  File "c:\Users\wjs31\.conda\envs\EXAM_MML\lib\site-packages\pandas\io\formats\html.py", line 88, in to_string
    lines = self.render()
  File "c:\Users\wjs31\.conda\envs\EXAM_MML\lib\site-packages\pandas\io\formats\html.py", line 644, in render
    super().render()
  File "c:\Users\wjs31\.conda\envs\EXAM_MML\lib\site-packages\pandas\io\formats\html.py", line 94, in render
    se

                       model       MAE      MAPE        R2  dif_mean  \
0           LinearRegression  0.026319  0.868144  0.032088 -0.000619   
0                      Lasso  0.026770  0.882421 -0.000218 -0.000605   
0                      Ridge  0.026319  0.868144  0.032088 -0.000619   
0              BayesianRidge  0.026318  0.868137  0.032093 -0.000619   
0                 ElasticNet  0.026770  0.882421 -0.000218 -0.000605   
0               SGDRegressor  0.026353  0.869405  0.031035 -0.000756   
0      RandomForestRegressor  0.020720  0.682446  0.411869 -0.000060   
0  GradientBoostingRegressor  0.025446  0.838909  0.154870 -0.000546   
0          AdaBoostRegressor  0.044197  1.450397 -1.108532  0.027930   
0              LGBMRegressor  0.024027  0.791635  0.282065 -0.000387   

    dif_std   dif_max   dif_min  dif_median  
0  0.040262  0.354901 -0.493781    0.000469  
0  0.040929  0.388265 -0.481735   -0.001735  
0  0.040262  0.354902 -0.493780    0.000469  
0  0.040262  0.354988 -

In [ ]:
# Scaler 없이 데이터 학습 및 분석
# 모델별 성능 확인
